In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install keras-rl
!pip install keras==2.2.4
!pip install tensorflow-gpu==1.13.1 

     |████████████████████████████████| 40kB 2.1MB/s 
  Created wheel for keras-rl: filename=keras_rl-0.4.2-cp36-none-any.whl size=48379 sha256=9c5abc625fc3d81bd59a5f940952c5485385703aac17fc7d91fb78bf19eae28e
  Stored in directory: /root/.cache/pip/wheels/7d/4d/84/9254c9f2e8f51865cb0dac8e79da85330c735551d31f73c894
Successfully built keras-rl
     |████████████████████████████████| 317kB 2.9MB/s 
  Found existing installation: Keras 2.2.5
    Uninstalling Keras-2.2.5:
      Successfully uninstalled Keras-2.2.5
     |████████████████████████████████| 345.2MB 35kB/s 
     |████████████████████████████████| 368kB 51.4MB/s 
     |████████████████████████████████| 3.2MB 30.7MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 1.13.1 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 1.13.0 which is incompatible.
  Found existing installation: tensorflow-estimator

In [3]:
from __future__ import division
import gym
import atari_py
from PIL import Image
import numpy as np
import keras
import tensorflow as tf

from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Convolution2D, Permute, Input, concatenate
from keras.optimizers import Adam
from keras.utils import plot_model
import keras.backend as K

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor

#print(atari_py.list_games())

print(keras.__version__)
print(tf.__version__)


Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn

2.2.4
1.13.1


In [0]:
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4

class AtariProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3  # (height, width, channel)
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert('L')  # resize and convert to grayscale
        processed_observation = np.array(img)
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('float32')/255 

    #def process_reward(self, reward):
     #   return np.clip(reward, -1., 1.)

In [0]:
#game = 'BeamRider-v0' #9
#game = 'Breakout-v0' #4
#game = 'Solaris-v0' #18
#game = 'DemonAttack-v0' #6
#game = 'AirRaid-v0' #6

game1 = 'AirRaid-v0' #6
game2 = 'Centipede-v0' #18

env1 = gym.make(game1, full_action_space=True)
np.random.seed(123)
env1.seed(123)

env2 = gym.make(game2, full_action_space=True)
np.random.seed(123)
env2.seed(123)


if (env1.action_space.n == env2.action_space.n):
  nb_actions = env1.action_space.n
else:
  raise RuntimeError('Actions does not match for 2 enviroments.')

In [6]:
input_shape = (WINDOW_LENGTH,) + INPUT_SHAPE


# CNN for environment 1
env1_input_layer = Input(shape=input_shape)
if K.image_dim_ordering() == 'tf':
    # (width, height, channels)
    env1_permute = Permute((2, 3, 1))(env1_input_layer)
elif K.image_dim_ordering() == 'th':
    # (channels, width, height)
    env1_permute = Permute((1, 2, 3))(env1_input_layer)
else:
    raise RuntimeError('Unknown image_dim_ordering for environment 1.')

env1_Conv_1 = Convolution2D(32, (8, 8), strides=(4, 4), activation = 'relu')(env1_permute)
env1_Conv_2 = Convolution2D(64, (4, 4), strides=(2, 2), activation = 'relu')(env1_Conv_1)
env1_Conv_3 = Convolution2D(64, (3, 3), strides=(1, 1), activation = 'relu')(env1_Conv_2)
env1_flatten = Flatten()(env1_Conv_3)
env1_dense = Dense(512, activation = 'relu')(env1_flatten)


# CNN for environment 2
env2_input_layer = Input(shape=input_shape)
if K.image_dim_ordering() == 'tf':
    # (width, height, channels)
    env2_permute = Permute((2, 3, 1))(env2_input_layer)
elif K.image_dim_ordering() == 'th':
    # (channels, width, height)
    env2_permute = Permute((1, 2, 3))(env2_input_layer)
else:
    raise RuntimeError('Unknown image_dim_ordering for environment 2.')
    
env2_Conv_1 = Convolution2D(32, (8, 8), strides=(4, 4), activation = 'relu')(env2_permute)
env2_Conv_2 = Convolution2D(64, (4, 4), strides=(2, 2), activation = 'relu')(env2_Conv_1)
env2_Conv_3 = Convolution2D(64, (3, 3), strides=(1, 1), activation = 'relu')(env2_Conv_2)
env2_flatten = Flatten()(env2_Conv_3)
env2_dense = Dense(512, activation = 'relu')(env2_flatten)

shared_layer = Dense(nb_actions, activation = 'linear', name = 'shared_layer')

env1_output = shared_layer(env1_dense)
env2_output = shared_layer(env2_dense)


model1 = Model(inputs = [env1_input_layer], outputs = [env1_output]) 
model2 =  Model(inputs = [env2_input_layer], outputs = [env2_output])

print(model1.summary())
plot_model(model1)
print(model2.summary())
plot_model(model2)


Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 4, 84, 84)         0         
_________________________________________________________________
permute_1 (Permute)          (None, 84, 84, 4)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 32)        8224      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 9, 64)          32832     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 7, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 3136)              0         
_________________________________________________________________
dens

In [0]:
memory = SequentialMemory(limit=1000000, window_length=WINDOW_LENGTH)
processor = AtariProcessor()
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.05, nb_steps=1000000)

dqn1 = DQNAgent(model=model1, nb_actions=nb_actions, policy=policy, memory=memory, processor=processor, nb_steps_warmup=50000, target_model_update=10000,train_interval=4)
dqn1.compile(Adam(lr=.00025), metrics=['mae'])


In [0]:
# from keras.callbacks import Callback as KerasCallback
# class save_at_episode(KerasCallback):
#     def on_episode_end(self, episode, logs={}):
#         self.model.save_weights('drive/My Drive/Colab Notebooks/Game Data/Breakout_weights_v1_{}'.format(episode), overwrite=True)
#         #print(" saved episode:{}".format(episode))
        

In [8]:
dqn1.fit(env1, nb_steps=300000, visualize=False, verbose=1) #callbacks=[save_at_episode()]
dqn1.save_weights('/content/drive/My Drive/Colab Notebooks/Game Data/DQN_model1_weights',overwrite=True)
#'/content/drive/My Drive/Colab Notebooks/Game Data/DQN_model1_weights'

Training for 300000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 29s 3ms/step - reward: 0.8500
13 episodes - episode_reward: 650.000 [150.000, 1900.000] - ale.lives: 0.999

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 26s 3ms/step - reward: 0.8475
10 episodes - episode_reward: 597.500 [200.000, 1275.000] - ale.lives: 0.999

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 26s 3ms/step - reward: 0.6050
12 episodes - episode_reward: 712.500 [25.000, 3325.000] - ale.lives: 0.999

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 25s 3ms/step - reward: 0.7075
13 episodes - episode_reward: 440.385 [200.000, 725.000] - ale.lives: 0.999

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 25s 3ms/step - reward: 0.6725
12 episodes - episode_reward: 643.750 [225.000, 1800.000] - ale.lives: 0.999

Interval 6 (50000 steps pe

In [9]:
#dqn1.load_weights('/content/drive/My Drive/Colab Notebooks/Game Data/DQN_model1_weights')
dqn1.test(env1, nb_episodes=10, visualize=False)

Testing for 10 episodes ...
Episode 1: reward: 25.000, steps: 204
Episode 2: reward: 200.000, steps: 489
Episode 3: reward: 400.000, steps: 466
Episode 4: reward: 100.000, steps: 322
Episode 5: reward: 100.000, steps: 384
Episode 6: reward: 125.000, steps: 301
Episode 7: reward: 225.000, steps: 378
Episode 8: reward: 350.000, steps: 259
Episode 9: reward: 75.000, steps: 317
Episode 10: reward: 75.000, steps: 283


In [10]:
import matplotlib.pyplot as plt
import seaborn as sns

shared_weights, shared_bias = model1.get_layer('shared_layer').weights
sns.distplot(K.eval(shared_weights).ravel())
plt.savefig('/content/drive/My Drive/Colab Notebooks/Game Data/shared_weights_before.jpg')
plt.clf()

<Figure size 432x288 with 0 Axes>

In [0]:
memory = SequentialMemory(limit=1000000, window_length=WINDOW_LENGTH)
processor = AtariProcessor()
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.05, nb_steps=1000000)

dqn2 = DQNAgent(model=model2, nb_actions=nb_actions, policy=policy, memory=memory, processor=processor, nb_steps_warmup=50000, target_model_update=10000,train_interval=4)
dqn2.compile(Adam(lr=.00025), metrics=['mae'])


In [12]:
dqn2.fit(env2, nb_steps=300000, visualize=False, verbose=1) #callbacks=[save_at_episode()]
dqn2.save_weights('/content/drive/My Drive/Colab Notebooks/Game Data/DQN_model2_weights',overwrite=True)

Training for 300000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 27s 3ms/step - reward: 2.7536
10 episodes - episode_reward: 2672.900 [1095.000, 6563.000] - ale.lives: 2.058

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 27s 3ms/step - reward: 2.3132
11 episodes - episode_reward: 2145.000 [857.000, 3913.000] - ale.lives: 2.027

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 27s 3ms/step - reward: 2.8787
10 episodes - episode_reward: 2901.000 [1085.000, 4963.000] - ale.lives: 2.038

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 27s 3ms/step - reward: 2.2839
11 episodes - episode_reward: 2027.727 [423.000, 2917.000] - ale.lives: 2.061

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 27s 3ms/step - reward: 1.9387
10 episodes - episode_reward: 1713.500 [246.000, 4775.000] - ale.lives: 2.134

Interval 6 (50000

In [21]:
dqn2.load_weights('/content/drive/My Drive/Colab Notebooks/Game Data/DQN_model2_weights')
dqn2.test(env2, nb_episodes=10, visualize=False)

Testing for 10 episodes ...
Episode 1: reward: 5328.000, steps: 1644
Episode 2: reward: 2230.000, steps: 973
Episode 3: reward: 7591.000, steps: 1193
Episode 4: reward: 3549.000, steps: 1038
Episode 5: reward: 3255.000, steps: 1012
Episode 6: reward: 3545.000, steps: 1313
Episode 7: reward: 3600.000, steps: 1102
Episode 8: reward: 10527.000, steps: 1932
Episode 9: reward: 5032.000, steps: 1081
Episode 10: reward: 4716.000, steps: 1471


In [14]:
shared_weights, shared_bias = model1.get_layer('shared_layer').weights
sns.distplot(K.eval(shared_weights).ravel())
plt.savefig('/content/drive/My Drive/Colab Notebooks/Game Data/shared_weights_after.jpg')
plt.clf()

<Figure size 432x288 with 0 Axes>

In [0]:
dqn1.save_weights('/content/drive/My Drive/Colab Notebooks/Game Data/DQN_model1_weights_changed',overwrite=True)

In [20]:
dqn1.load_weights('/content/drive/My Drive/Colab Notebooks/Game Data/DQN_model1_weights_changed')
dqn1.test(env1, nb_episodes=10, visualize=False)

Testing for 10 episodes ...
Episode 1: reward: 300.000, steps: 253
Episode 2: reward: 125.000, steps: 208
Episode 3: reward: 300.000, steps: 253
Episode 4: reward: 300.000, steps: 245
Episode 5: reward: 175.000, steps: 195
Episode 6: reward: 300.000, steps: 251
Episode 7: reward: 300.000, steps: 249
Episode 8: reward: 175.000, steps: 197
Episode 9: reward: 300.000, steps: 244
Episode 10: reward: 300.000, steps: 245


In [0]:
#env.close()